In [25]:
import sklearn
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from PIL import Image, ImageOps
from skimage import feature

from collections import OrderedDict
import statistics

In [26]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [27]:
# The following image was fed through a high pass filter (std 4), sobel edge detection, and a random threshholding point (approx 56)
im = Image.open("./HiPass_sobel_Threshold.tif")
im_gray = ImageOps.grayscale(im)
imarray = np.array(im_gray)

In [28]:
im_gray.show()

In [29]:
X, Y = imarray.shape

In [30]:
X * Y

1228800

In [31]:
im.show()

In [32]:
0 in imarray[:2]

False

In [33]:
unvisited = set([(x, y) for x in range(2, X) for y in range(3, Y) if imarray[x, y] == 255])
seeds = []

def get_cardinal_nodes(node, X, Y):
    x = node[0]
    y = node[1]
    
    result = [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]
    
    return [(i, j) for (i, j) in result if (i >= 0 and i < X and j >= 0 and j < Y)]


components = {}
count = 0

while unvisited:
    current_node = unvisited.pop()
    seeds.append(current_node)
    queue = [current_node]
    
    min_x = current_node[0]
    max_x = current_node[0]
    min_y = current_node[1]
    max_y = current_node[1]
    
    while queue:
        next_node = queue.pop()
        
        potential_neighbor_nodes = get_cardinal_nodes(next_node, X, Y)
        
        for node in potential_neighbor_nodes:
            i, j = node
            
            if node in unvisited:
                unvisited.remove(node)
                
                min_x = min(i, min_x)
                max_x = max(i, max_x)
                min_y = min(j, min_y)
                max_y = max(j, max_y)
                
                queue.append(node)
                
    components[count] = {"min_x": min_x, "max_x": max_x, "min_y": min_y, "max_y": max_y}
    count += 1


In [34]:
components

{0: {'min_x': 436, 'max_x': 437, 'min_y': 677, 'max_y': 680},
 1: {'min_x': 336, 'max_x': 386, 'min_y': 111, 'max_y': 167},
 2: {'min_x': 435, 'max_x': 446, 'min_y': 1012, 'max_y': 1025},
 3: {'min_x': 135, 'max_x': 164, 'min_y': 435, 'max_y': 460},
 4: {'min_x': 650, 'max_x': 653, 'min_y': 1236, 'max_y': 1255},
 5: {'min_x': 251, 'max_x': 258, 'min_y': 897, 'max_y': 911},
 6: {'min_x': 236, 'max_x': 382, 'min_y': 188, 'max_y': 319},
 7: {'min_x': 486, 'max_x': 507, 'min_y': 82, 'max_y': 88},
 8: {'min_x': 73, 'max_x': 143, 'min_y': 49, 'max_y': 117},
 9: {'min_x': 535, 'max_x': 569, 'min_y': 1085, 'max_y': 1152},
 10: {'min_x': 290, 'max_x': 320, 'min_y': 197, 'max_y': 236},
 11: {'min_x': 364, 'max_x': 398, 'min_y': 659, 'max_y': 711},
 12: {'min_x': 330, 'max_x': 352, 'min_y': 481, 'max_y': 500},
 13: {'min_x': 716, 'max_x': 751, 'min_y': 144, 'max_y': 172},
 14: {'min_x': 699, 'max_x': 737, 'min_y': 462, 'max_y': 501},
 15: {'min_x': 104, 'max_x': 104, 'min_y': 98, 'max_y': 102},
 

In [35]:
# Create a dictionary with Bounding Box Areas as keys and counts of components with said area as values
area_filter = OrderedDict()

for dim in components.values():
    width, height = dim['max_x'] - dim['min_x'] + 1, dim['max_y'] - dim['min_y'] + 1
    area = width * height

    area_filter[area] = area_filter.get(area, 0) + 1

area_filter = OrderedDict(sorted(area_filter.items()))

area_filter

OrderedDict([(1, 2568),
             (2, 1141),
             (3, 384),
             (4, 474),
             (5, 60),
             (6, 384),
             (7, 11),
             (8, 212),
             (9, 110),
             (10, 120),
             (11, 3),
             (12, 227),
             (14, 39),
             (15, 113),
             (16, 76),
             (18, 99),
             (20, 93),
             (21, 46),
             (22, 6),
             (24, 104),
             (25, 37),
             (27, 14),
             (28, 61),
             (30, 73),
             (32, 40),
             (33, 17),
             (35, 33),
             (36, 64),
             (38, 1),
             (39, 6),
             (40, 44),
             (42, 36),
             (44, 25),
             (45, 22),
             (48, 47),
             (49, 16),
             (50, 15),
             (52, 4),
             (54, 23),
             (55, 20),
             (56, 33),
             (57, 1),
             (60, 28),
             

In [36]:
areas, counts = np.array(list(area_filter.keys())), np.array(list(area_filter.values()))

weighted_average = np.inner(areas, counts) / sum(counts)

weighted_average

86.98991371977154

In [38]:
# Grow components from seeds

def grow_seeds(image, seeds, color=255):

    X, Y = image.shape
    components = {}

    while seeds:
        seed = seeds.pop(0)
        queue = [seed]
        curr_component = set()
        curr_component.add(seed)

        while queue:
            curr_node = queue.pop(0)
            neighbors = get_cardinal_nodes(curr_node, X, Y)

            for neighbor in neighbors:
                i, j = neighbor

                if neighbor not in curr_component and image[i, j] == color:
                    curr_component.add(neighbor)
                    queue.append(neighbor)


        # Save found component
        components[seed] = curr_component

    return components

all_components = grow_seeds(imarray, seeds)

In [44]:
min_pixels = 100

filtered_components = {k:v for k,v in all_components.items() if len(v) >= min_pixels}

In [46]:
len(filtered_components.keys())

537